# Real project starts from here:-

In [ ]:
!pip install segment-anything opencv-python

In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# # 2. Install Dependencies
# print("⏳ Installing libraries...")
# !pip install segment-anything opencv-python tqdm
# # Download SAM Checkpoint if missing
# if not os.path.exists("sam_vit_b_01ec64.pth"):
#     !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
# print("✅ Ready.")

Mounted at /content/drive


In [ ]:
# ==========================================
# 🚀 CONTROLNET PROJECT INITIALIZER (Update)
# ==========================================
import os

PROJECT_ROOT = "/content/drive/MyDrive/ControlNet_Project"
os.makedirs(f"{PROJECT_ROOT}/src", exist_ok=True)
os.makedirs(f"{PROJECT_ROOT}/data/processed", exist_ok=True)
os.makedirs(f"{PROJECT_ROOT}/metadata", exist_ok=True)

# 1. WRITE STRATEGIES (The Brains)
strategies_code = """
import cv2
import numpy as np
import torch
import os
from abc import ABC, abstractmethod
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

class ProcessingStrategy(ABC):
    @abstractmethod
    def load_model(self): pass
    @abstractmethod
    def generate_control_signal(self, image_path): pass

class CarStrategy(ProcessingStrategy):
    '''
    Multi-Component Palette Strategy:
    1. Finds all parts (Body, Wheels, Windows).
    2. Filters out the background.
    3. Colors each part with its REAL average color from the photo.
    4. Adds Canny edges for structure.
    '''
    def __init__(self, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.mask_generator = None
        self.colors = {"edges": (255, 255, 255)}

    def load_model(self):
        print(f"🔹 [CarStrategy] Loading SAM on {self.device}...")
        checkpoint_path = "sam_vit_b_01ec64.pth"
        if not os.path.exists(checkpoint_path):
             if os.path.exists(f"../{checkpoint_path}"): checkpoint_path = f"../{checkpoint_path}"
             else: os.system("wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth")

        sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
        sam.to(device=self.device)
        self.mask_generator = SamAutomaticMaskGenerator(
            model=sam, points_per_side=32, pred_iou_thresh=0.86, stability_score_thresh=0.92,
            crop_n_layers=1, crop_n_points_downscale_factor=2, min_mask_region_area=100
        )

    def get_dominant_color(self, image, mask):
        mean_color = cv2.mean(image, mask=mask.astype(np.uint8))[:3]
        return (int(mean_color[0]), int(mean_color[1]), int(mean_color[2]))

    def generate_control_signal(self, image_path):
        image = cv2.imread(image_path)
        if image is None: return None
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        masks = self.mask_generator.generate(image_rgb)
        masks = sorted(masks, key=(lambda x: x['area']), reverse=False)
        control_map = np.zeros_like(image)

        for m in masks:
            seg = m['segmentation']
            touches_border = (np.any(seg[0, :]) or np.any(seg[-1, :]) or np.any(seg[:, 0]) or np.any(seg[:, -1]))
            is_too_big = m['area'] > (image.shape[0] * image.shape[1] * 0.90)

            if not touches_border and not is_too_big:
                part_color = self.get_dominant_color(image, seg)
                control_map[seg] = part_color

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        control_map[edges > 0] = self.colors["edges"]
        return control_map

class SneakerStrategy(CarStrategy):
    pass
"""
with open(f"{PROJECT_ROOT}/src/strategies.py", "w") as f:
    f.write(strategies_code)

# 2. WRITE BATCH PROCESSOR (Now with Resume Capability)
batch_code = """
import os
import cv2
import json
from tqdm import tqdm
from src.strategies import CarStrategy, SneakerStrategy

def process_batch(input_dir, output_dir, manifest_path, mode="car", limit=None):
    # 1. Setup
    if not os.path.exists(output_dir): os.makedirs(output_dir)

    # Select Strategy
    if mode == "car": strategy = CarStrategy()
    elif mode == "sneaker": strategy = SneakerStrategy()
    else: raise ValueError("Unknown mode")

    # Load model only if we actually have work to do
    model_loaded = False

    # 2. List Images
    valid_exts = (".jpg", ".jpeg", ".png", ".webp")
    all_files = [f for f in os.listdir(input_dir) if f.lower().endswith(valid_exts)]
    all_files.sort() # Sort ensures deterministic order for resuming

    if limit: all_files = all_files[:limit]

    print(f"🚀 Found {len(all_files)} images. Checking for existing progress...")

    valid_entries = []
    skipped_count = 0
    processed_count = 0

    # 3. Process Loop
    for filename in tqdm(all_files):
        img_path = os.path.join(input_dir, filename)

        # Define expected output filename
        save_name = os.path.splitext("control_" + filename)[0] + ".png"
        save_path = os.path.join(output_dir, save_name)

        # --- CHECKPOINT LOGIC ---
        # If the file already exists, we skip the GPU processing!
        if os.path.exists(save_path):
            skipped_count += 1
            # We still add it to the valid_entries list so the JSONL is complete
            entry = {
                "text": f"A photo of a {mode}",
                "image": os.path.abspath(img_path),
                "conditioning_image": os.path.abspath(save_path)
            }
            valid_entries.append(entry)
            continue
        # ------------------------

        # Lazy Loading: Only load SAM if we actually need to process a new image
        if not model_loaded:
            print("⚙️ New images found. Loading SAM model into GPU...")
            strategy.load_model()
            model_loaded = True

        try:
            control_signal = strategy.generate_control_signal(img_path)

            if control_signal is not None:
                cv2.imwrite(save_path, control_signal)
                processed_count += 1

                entry = {
                    "text": f"A photo of a {mode}",
                    "image": os.path.abspath(img_path),
                    "conditioning_image": os.path.abspath(save_path)
                }
                valid_entries.append(entry)

        except Exception as e:
            print(f"❌ Failed {filename}: {e}")

    # 4. Save Manifest (JSONL)
    print(f"📝 Writing manifest with {len(valid_entries)} pairs ({skipped_count} skipped, {processed_count} new)...")
    with open(manifest_path, "w") as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + "\\n")

    print(f"✅ Batch Complete! Manifest saved to {manifest_path}")
"""

with open(f"{PROJECT_ROOT}/batch_processor.py", "w") as f:
    f.write(batch_code)
print("✅ Updated: batch_processor.py (Resume Mode Enabled)")

✅ Updated: batch_processor.py (Resume Mode Enabled)


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/ControlNet_Project")
from batch_processor import process_batch

# Run!
process_batch(
    input_dir="/content/drive/MyDrive/ControlNet_Project/data/raw/cars",
    output_dir="/content/drive/MyDrive/ControlNet_Project/data/processed",
    manifest_path="/content/drive/MyDrive/ControlNet_Project/metadata/train.jsonl",
    mode="car",
    limit=2000  # <--- Processing only 2000 of your 8000 images
)

🚀 Found 2000 images. Checking for existing progress...


  0%|          | 0/2000 [00:00<?, ?it/s]

⚙️ New images found. Loading SAM model into GPU...
🔹 [CarStrategy] Loading SAM on cuda...


 53%|█████▎    | 1063/2000 [2:46:41<3:12:27, 12.32s/it]

❌ Failed 01063.jpg: CUDA out of memory. Tried to allocate 6.35 GiB. GPU 0 has a total capacity of 14.74 GiB of which 6.26 GiB is free. Process 253267 has 8.48 GiB memory in use. Of the allocated memory 7.63 GiB is allocated by PyTorch, and 737.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


 68%|██████▊   | 1360/2000 [3:50:25<2:07:49, 11.98s/it]

In [ ]:
import os
from google.colab import drive

# 1. Check if Drive is Mounted
if not os.path.exists('/content/drive'):
    print("❌ Google Drive is NOT mounted.")
    print("   👉 Fix: Run `drive.mount('/content/drive')` first.")
else:
    print("✅ Google Drive is mounted.")

# 2. Check Project Root
PROJECT_ROOT = "/content/drive/MyDrive/ControlNet_Project"
if not os.path.exists(PROJECT_ROOT):
    print(f"❌ Project Root missing: {PROJECT_ROOT}")
    print("   👉 Fix: Run the 'Project Initializer' cell again to create folders.")
else:
    print(f"✅ Project Root found.")

# 3. Check Input Data (Crucial)
RAW_DATA = f"{PROJECT_ROOT}/data/raw/cars"
if not os.path.exists(RAW_DATA):
    print(f"❌ Raw Data folder missing: {RAW_DATA}")
    print("   👉 Fix: Check if you named the folder 'Car' instead of 'cars' (case sensitive!)")
else:
    # Check for images
    images = os.listdir(RAW_DATA)
    if len(images) == 0:
        print(f"⚠️ Folder exists but is EMPTY: {RAW_DATA}")
        print("   👉 Fix: Upload your images here!")
    else:
        print(f"✅ Found {len(images)} images in raw/cars. (First file: {images[0]})")

# 4. Check for the Script
SCRIPT_PATH = f"{PROJECT_ROOT}/batch_processor.py"
if not os.path.exists(SCRIPT_PATH):
    print(f"❌ Script missing: {SCRIPT_PATH}")
    print("   👉 Fix: Re-run the cell that writes 'batch_processor.py'.")
else:
    print(f"✅ batch_processor.py exists.")

# 5. Check for SAM Model
if not os.path.exists("sam_vit_b_01ec64.pth") and not os.path.exists(f"{PROJECT_ROOT}/sam_vit_b_01ec64.pth"):
    print("❌ SAM Model checkpoint missing.")
    print("   👉 Fix: Run `!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth`")
else:
    print("✅ SAM Checkpoint found.")

In [ ]:
!zip -r /content/drive/MyDrive/processed_images.zip /content/drive/MyDrive/ControlNet_Project/data/processed

  adding: content/drive/MyDrive/ControlNet_Project/data/processed/ (stored 0%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00365.png (deflated 40%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00366.png (deflated 39%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00367.png (deflated 37%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00368.png (deflated 29%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00369.png (deflated 44%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00370.png (deflated 41%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00371.png (deflated 36%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00372.png (deflated 37%)
  adding: content/drive/MyDrive/ControlNet_Project/data/processed/control_00373.png (deflated 38%)
  adding: content/drive/MyDriv